In [1]:
import numpy as np
from mostlyai import MostlyAI

In [2]:
# initialize client
mostly = MostlyAI(api_key='mostly-adcd77a4ca7523519dc19b4ec9c9a7f6e0b2e37e16599b4703b03944dc824ed2', base_url='https://app.mostly.ai')

In [3]:
# fetch configuration via API
g = mostly.generators.get('6a19773e-9bd7-402a-aa65-a75e1e688c26')
config = g.config()
config

{'name': 'cleaned_ipums_data',
 'description': None,
 'tables': [{'name': 'cleaned_ipums_data',
   'sourceConnectorId': 'ac9bdefa-05d9-4c98-bc55-f1740854a025',
   'location': 'mostly-production-mostly-s3-bucket-mostly-app/connectors/ac9bdefa-05d9-4c98-bc55-f1740854a025/cleaned_ipums_data/cleaned_ipums_data.csv',
   'data': None,
   'modelConfiguration': {'maxSampleSize': 197756,
    'batchSize': None,
    'modelSize': 'M',
    'maxTrainingTime': 10,
    'maxEpochs': 100,
    'maxSequenceWindow': 100,
    'enableFlexibleGeneration': False,
    'valueProtection': False,
    'rareCategoryReplacementMethod': 'CONSTANT',
    'compute': 'd5f093a6-208f-4b79-92c6-6d50d02d8c8c'},
   'textModelConfiguration': None,
   'primaryKey': None,
   'foreignKeys': None,
   'columns': [{'name': 'incwage', 'modelEncodingType': 'NUMERIC_BINNED'},
    {'name': 'years_of_educ', 'modelEncodingType': 'NUMERIC_DISCRETE'},
    {'name': 'potential_experience', 'modelEncodingType': 'NUMERIC_BINNED'},
    {'name': '

In [4]:
# probe the generator for some samples
mostly.probe('6a19773e-9bd7-402a-aa65-a75e1e688c26', size=10)

,incwage,years_of_educ,potential_experience,non_white,sex
0,0.606523,-0.044869,0.289084,0,1
1,-0.530665,-0.044869,0.287699,1,0
2,0.019349,0.339777,0.631867,0,1
3,2.329620,0.724424,1.933060,0,0
4,-0.413830,-0.429516,-0.277415,0,1
5,-0.546100,-0.429516,1.027519,0,0
6,-0.534047,-0.044869,-0.192611,0,0
7,-0.068944,-0.429516,-0.351887,0,0
8,-0.392852,-0.044869,-1.410815,0,0
9,-0.522780,-1.198810,0.140877,0,0


We need 20 synthetic data sets. Original data set size was 197,756, so we need $197,756 \times 20 = 3,955,120$ samples.

Due to limits on the free version, we will break it into four requests. This will be done manually over 4 days.

In [5]:
# use generator to create a synthetic dataset
sd = mostly.generate('6a19773e-9bd7-402a-aa65-a75e1e688c26', size=988_780)
sd.data()

Created synthetic dataset ]8;id=729178;https://app.mostly.ai/d/synthetic-datasets/df2bb30e-104a-4b39-8572-b67b6e77e255\df2bb30e-104a-4b39-8572-b67b6e77e255]8;;\ with generator ]8;id=488348;https://app.mostly.ai/d/generators/6a19773e-9bd7-402a-aa65-a75e1e688c26\6a19773e-9bd7-402a-aa65-a75e1e688c26]8;;\

Started synthetic dataset generation

Output()

🎉 Your synthetic dataset is ready! Use it to consume the generated data. Share it so others can do the same.

,incwage,years_of_educ,potential_experience,non_white,sex
0,0.290532,1.109071,1.938983,0,0
1,1.76231,1.109071,0.047618,0,0
2,-0.242653,1.109071,0.127913,0,1
3,-0.655865,-0.429516,1.448171,1,1
4,-0.215971,1.777461,-1.232504,0,1
...,...,...,...,...,...
9982,-0.262373,-1.19881,-1.084325,0,0
9983,1.272854,1.777461,1.08297,0,1
9984,-0.268793,-2.061237,1.714525,0,0
9985,-0.705486,-0.044869,-0.520967,0,1


Split the generated synthetic data into 20 data sets.

In [6]:
split_sd = np.array_split(sd.data(), 5)

C:\ProgramData\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
split_sd[0]

,incwage,years_of_educ,potential_experience,non_white,sex
0,0.290532,1.109071,1.938983,0,0
1,1.76231,1.109071,0.047618,0,0
2,-0.242653,1.109071,0.127913,0,1
3,-0.655865,-0.429516,1.448171,1,1
4,-0.215971,1.777461,-1.232504,0,1
...,...,...,...,...,...
7985,2.225233,1.109071,0.472216,0,0
7986,1.326913,-0.044869,1.042425,0,0
7987,-0.494483,1.109071,0.616946,0,1
7988,-0.226832,-0.044869,-1.17096,0,1


# MUST CHANGE INDEX WHEN SAVING ADDITIONAL DATASETS SO YOU DON'T OVERWRITE EXISTING

In [8]:
for i, sX in enumerate(split_sd):
    sX.to_csv("../Data/IPUMS/Synthetic Datasets/mostly_" + str(i) + ".csv", index=False)